In [1]:
import cv2
import gc
import numpy as np
import matplotlib.pyplot as plt
import dicom as pdicom
import os
import pandas as pd
import numpy as np
import random


from PIL import Image
from collections import Counter
from natsort import natsorted

import tensorflow as tf
import keras
import keras.backend as K
from keras.preprocessing.image import array_to_img, img_to_array, load_img

### postavke za konfiguraciju
config = tf.ConfigProto( device_count = {'GPU': 2 , 'CPU': 2} ) 
sess = tf.Session(config=config) 
keras.backend.set_session(sess)

/home/ibrahimhadzic45/anaconda3/envs/tf/lib/python3.6/site-packages/dicom/__init__.py:53: UserWarning: 
This code is using an older version of pydicom, which is no longer 
maintained as of Jan 2017.  You can access the new pydicom features and API 
by installing `pydicom` from PyPI.
See 'Transitioning to pydicom 1.x' section at pydicom.readthedocs.org 
for more information.

  warnings.warn(msg)
Using TensorFlow backend.


In [2]:
from keras.models import load_model
loaded_model = load_model('varian_CT_cnn.h5')

In [15]:
## list all dicom images

# Set the path of the test data
dicom_path = "VarianDataset"
lstFilesDCM = []  # create an empty list
for dirName, subdirList, fileList in os.walk(dicom_path):
    for filename in fileList:
        if ".dcm" in filename.lower():  # check whether the file's DICOM
            lstFilesDCM.append(os.path.join(dirName,filename))

In [16]:
arr = []
d = dict()

# loop through all the DICOM files
for filenameDCM in lstFilesDCM:
    if "Image" not in filenameDCM:
        continue
    elif "CT" not in filenameDCM:
        continue
    
    # read the file
    ds = pdicom.read_file(filenameDCM)
    arr.append(ds.pixel_array.shape)
    shape = ds.pixel_array.shape
    
    if shape in d.keys():
        d[shape].append(filenameDCM)
    else:
        d[shape] = [filenameDCM]

print(len(arr))
Counter(arr)

2860


Counter({(512, 512): 2860})

In [17]:
# sorting the images the way they were scanned
for key, value in d.items():
    d[key] = natsorted(value)

In [18]:
# Resizing with maintaining aspect ratio and adding padding if necessary

def resize_pad_image(path):
    im = pdicom.read_file(path).pixel_array
    im = array_to_img(im.reshape(len(im), len(im[0]), -1))
    
    IMG_SIZE = 224      

    new_size   = tuple([int(x*(float(IMG_SIZE)/max(im.size))) for x in im.size])
    im_resized = im.resize(new_size, Image.ANTIALIAS)
    new_im     = Image.new("L", (IMG_SIZE, IMG_SIZE))
    new_im.paste(im_resized, 
                 ((IMG_SIZE-new_size[0])//2,
                  (IMG_SIZE-new_size[1])//2))

    return np.array(new_im).reshape(224, 224, 1)

In [19]:
data = []
for key, value in d.items():
    for img_path in natsorted(value):
        data.append([img_path, resize_pad_image(img_path)])

In [78]:
slika = data[17][1]
slika = np.stack((slika.reshape(224,224),)*3, -1) # making an RGB out of grayscale
slika = slika.reshape(-1, 224, 224, 3)

In [79]:
pred = loaded_model.predict(slika)
pred_class = pred.argmax(axis=-1)
#pred_label = sorted(labels)[y_classes]

In [80]:
pred_classes

array([0])

In [86]:
for elem in data:
    img = elem[1]
    img = np.stack((img.reshape(224,224),)*3, -1)
    img = img.reshape(-1, 224, 224, 3)
    pred = loaded_model.predict(img)
    pred_class = pred.argmax(axis=-1)
    print(pred_classes)
    


[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]


[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
